# <b>Avalia o comportamento das Requests</b>

In [1]:
# Importando as dependências
from quantumnet.components import Controller, Network
from quantumnet.objects import time
from simulations import *
logger.deactivate()

Informações a serem coletadas

In [2]:
received_requests = {}
successful_requests = {}
fulfilled_requests = {}

### <b>Preenchimento Proativo</b>

In [3]:
# Parâmetros das simulações
network_info = {
    'n_initial_qubits': 10,
    'n_initial_eprs': 10,
    'topology_name': 'Grade',
    'topology_params': (3, 4),
    'time_to_refill': 10,
}
controller_info = {
    'default_ttl': 50,
}
request_info = {
    'num_hosts': 12,
    'n_requests': 100,
    'traffic_type': 'random',
    'burst_probability': 0.3,
    'burst_size': 25,
    'requests_per_burst': 5,
    'fmin_range': (0.5, 1.0),
    'eprs_range': (1, 10),
}
# Exclusivo para preenchimento proativo e hibrido
proactive_params = {
    1 : { 'frange': (0.6, 0.9), 'neprs': 10 },
}

In [4]:
# Inicializando a simulação
sim = Sim(network_info, controller_info, request_info)
sim.requests = generate_traffic(request_info)

print(f"Requests: {sim.requests}")

# Preenche as tabelas de forma proativa
for alice in sim.network.hosts:
    print(f"Adicionando regras para {alice}")
    for bob in sim.network.hosts:
        for key in proactive_params.keys():
            sim.controller.add_match_route_rule_in_host_proactive(alice, bob, proactive_params[key]['frange'], proactive_params[key]['neprs'])
    # Mostra as tabelas
    sim.network.get_host(alice).draw_flow_table()

Requests: [R12AA20:11-5:0.92:2, R5550A0:7-11:0.86:3, R12ACC0:5-3:0.81:2, R12ACF0:6-10:0.69:7, R12AD20:10-3:0.53:10, R12AD50:11-3:0.89:9, R12AD80:11-3:0.92:2, R12ADB0:4-10:0.87:4, R12ADE0:1-6:0.7:2, R12AE10:11-6:0.73:10, R12AE40:1-7:0.94:6, R12AE70:10-5:1.0:4, R12AEA0:3-1:0.64:8, R12AED0:6-11:0.73:4, R12AF00:7-9:0.74:10, R12AF30:3-6:0.7:6, R12AF60:1-3:0.76:6, R12AF90:1-11:0.74:6, R12AFC0:6-2:0.92:4, R12AFF0:5-6:0.57:5, R12B020:1-8:0.74:8, R12B050:4-8:0.87:2, R12B080:1-8:0.97:3, R12B0B0:6-1:0.82:4, R12B0E0:7-9:0.54:6, R12B110:3-10:0.78:7, R12B140:3-11:0.59:1, R12B170:0-2:0.85:3, R12B1A0:11-3:0.88:7, R12B1D0:6-0:0.64:1, R12B200:6-10:0.73:8, R12B230:1-7:0.88:3, R12B260:2-0:0.81:3, R12B290:8-9:0.99:3, R12B2C0:7-6:0.78:8, R12B2F0:7-3:0.68:2, R12B320:2-0:0.68:6, R12B350:1-3:0.96:7, R12B380:7-0:0.7:3, R12B3B0:10-0:0.94:7, R12B3E0:3-4:0.54:3, R12B410:7-6:0.8:5, R12B440:4-9:0.65:3, R12B470:9-2:0.57:6, R12B4A0:6-0:0.76:2, R12B4D0:3-7:0.72:5, R12B500:2-11:0.58:5, R12B530:0-10:0.79:8, R12B560:5-0:0

In [5]:
# Processa as requisições
for request in sim.requests:
    print(f"[Time {time.get_current_time()}] Processando requisição {request}.")
    alice = sim.network.get_host(request.alice)
    rule = alice.find_rule_by_request(request)

    if rule == False:  # Caso não exista um match na tabela
        print(f"[Time {time.get_current_time()}] Descartando requisição {request} no Host {alice}.")
        request.starttime = time.get_current_time()
        request.endtime = time.get_current_time()
        # Registra a requisição descartada
        sim.controller.fulfill_request(request)
        
    else:  # Caso já exista a regra
        print(f"[Time {time.get_current_time()}] Atendendo requisição {request} no Host {alice}.")
        request.starttime = time.get_current_time()
        sim.controller.run_rule(rule[1])
        request.endtime = time.get_current_time() + 1        
        # Registra a requisição atendida
        sim.controller.successful_request(request)
        # Exibir informações da requisição
        print(f"Request {request}: Start Time = {request.starttime}, End Time = {request.endtime}")
        
    # Atualiza o tempo
    sim.update_time(1)

# Salva as requisições
received_requests['proativo'] = sim.requests.copy()
successful_requests['proativo'] = sim.controller.successful_requests
fulfilled_requests['proativo'] = sim.controller.fulfilled_requests

# Fim da simulação
time.reset()


[Time 0] Processando requisição R12AA20:11-5:0.92:2.
[Time 0] Descartando requisição R12AA20:11-5:0.92:2 no Host 11.


AttributeError: 'Sim' object has no attribute 'filename'

### <b>Preenchimento Reativo</b>

In [6]:
# Parâmetros das simulações
network_info = {
    'n_initial_qubits': 10,
    'n_initial_eprs': 10,
    'topology_name': 'Grade',
    'topology_params': (3, 4),
    'time_to_refill': 10,
}
controller_info = {
    'default_ttl': 50,
}
request_info = {
    'num_hosts': 12,
    'n_requests': 100,
    'traffic_type': 'random',
    'burst_probability': 0.3,
    'burst_size': 25,
    'requests_per_burst': 5,
    'fmin_range': (0.5, 1.0),
    'eprs_range': (1, 10),
}

In [ ]:
# Inicializando a simulação
sim = Sim(network_info, controller_info, request_info)
print("Requests:", sim.requests)

for request in sim.requests:
        print(f"[Time {time.get_current_time()}] Processando requisição {request}...")
        alice = sim.network.get_host(request.alice)
        rule = alice.find_rule_by_request(request)

        if rule == False:  # Caso não exista um match na tabela
            request.starttime = time.get_current_time()
            sim.update_time(3)
            alice.draw_flow_table()
            print(f"[Time {time.get_current_time()}] Adicionando regra no Host {alice}")
            sim.controller.add_match_route_rule_in_host_reactive(request)
            alice.draw_flow_table()
            rule = alice.find_rule_by_request(request)
            
        else:  # Caso já exista a regra
            print(f"[Time {time.get_current_time()}] Regra existente para {request} no Host {alice}.")
            request.starttime = time.get_current_time()
            sim.update_time(1)
                
        # Executa a regra
        print(f"[Time {time.get_current_time()}] Atendendo requisição {request} no Host {alice}.")
        sim.controller.run_rule(rule[1])
        request.endtime = time.get_current_time()
        # Registra a requisição atendida
        sim.controller.successful_request(request)
        
        # Exibir informações da requisição
        print(f"[Time {time.get_current_time()}] Request {request}: Start Time = {request.starttime}, End Time = {request.endtime}")

# Salva as requisições
received_requests['reativo'] = sim.requests.copy()
successful_requests['reativo'] = sim.controller.successful_requests
fulfilled_requests['reativo'] = sim.controller.fulfilled_requests

# Fim da simulação
time.reset()

### <b>Preenchimento Híbrido</b>

In [8]:
# Parâmetros das simulações
network_info = {
    'n_initial_qubits': 10,
    'n_initial_eprs': 10,
    'topology_name': 'Grade',
    'topology_params': (3, 4),
    'time_to_refill': 10,
}
controller_info = {
    'default_ttl': 50,
}
request_info = {
    'num_hosts': 12,
    'n_requests': 100,
    'traffic_type': 'random',
    'burst_probability': 0.3,
    'burst_size': 25,
    'requests_per_burst': 5,
    'fmin_range': (0.5, 1.0),
    'eprs_range': (1, 10),
}
# Exclusivo para preenchimento proativo e hibrido
proactive_params = {
    1 : { 'frange': (0.6, 0.7), 'neprs': 10 },
    5:  { 'frange': (0.8, 0.9), 'neprs': 10 },
}

In [ ]:
# Inicializando a simulação
sim = Sim(network_info, controller_info, request_info)

# Preenche as tabelas de forma proativa
for alice in sim.network.hosts:
    print(f"Adicionando regras para {alice}")
    for bob in sim.network.hosts:
        for key in proactive_params.keys():
            sim.controller.add_match_route_rule_in_host_proactive(alice, bob, proactive_params[key]['frange'], proactive_params[key]['neprs'])
    # Mostra as tabelas
    sim.network.get_host(alice).draw_flow_table()

In [ ]:
# Inicializando a simulação
sim = Sim(network_info, controller_info, request_info)
print("Requests:", sim.requests)

for request in sim.requests:
        print(f"[Time {time.get_current_time()}] Processando requisição {request}...")
        alice = sim.network.get_host(request.alice)
        rule = alice.find_rule_by_request(request)

        if rule == False:  # Caso não exista um match na tabela
            request.starttime = time.get_current_time()
            sim.update_time(3)
            alice.draw_flow_table()
            print(f"[Time {time.get_current_time()}] Adicionando regra no Host {alice}")
            sim.controller.add_match_route_rule_in_host_reactive(request)
            alice.draw_flow_table()
            rule = alice.find_rule_by_request(request)
            
        else:  # Caso já exista a regra
            print(f"[Time {time.get_current_time()}] Regra existente para {request} no Host {alice}.")
            request.starttime = time.get_current_time()
            sim.update_time(1)
                
        # Executa a regra
        print(f"[Time {time.get_current_time()}] Atendendo requisição {request} no Host {alice}.")
        sim.controller.run_rule(rule[1])
        request.endtime = time.get_current_time()
        # Registra a requisição atendida
        sim.controller.successful_request(request)
        
        # Exibir informações da requisição
        print(f"[Time {time.get_current_time()}] Request {request}: Start Time = {request.starttime}, End Time = {request.endtime}")

# Salva as requisições
received_requests['hibrido'] = sim.requests.copy()
successful_requests['hibrido'] = sim.controller.successful_requests
fulfilled_requests['hibrido'] = sim.controller.fulfilled_requests

# Fim da simulação
time.reset()

In [ ]:
import matplotlib.pyplot as plt

def calcular_percentual_atendimento(requisicoes_recebidas, requisicoes_atendidas):
    # Cálculo do percentual de atendimento por modo
    percentual_atendido = {}
    for modo in requisicoes_recebidas:
        total_recebidas = len(requisicoes_recebidas[modo])
        total_atendidas = len(requisicoes_atendidas[modo])
        percentual_atendido[modo] = (total_atendidas / total_recebidas) * 100 if total_recebidas > 0 else 0
    return percentual_atendido

percentual = calcular_percentual_atendimento(received_requests, successful_requests)

# Plotagem do gráfico
plt.figure(figsize=(8, 6))
plt.bar(percentual.keys(), percentual.values(), color=['#4c72b0', '#55a868', '#c44e52'])

plt.xlabel('Modo de Preenchimento')
plt.ylabel('Percentual de Requisições Atendidas (%)')
plt.ylim(0, 100)

# Exibe o valor de cada barra
for i, v in enumerate(percentual.values()):
    plt.text(i, v + 1, f'{v:.1f}%', ha='center')

plt.show()


In [ ]:
def plotar_latencia(requisicoes_por_modo):
    latencias = {'reativo': [], 'proativo': [], 'hibrido': []}
    
    for modo, requisicoes in requisicoes_por_modo.items():
        for request in requisicoes:
            latencia = request.endtime - request.starttime
            latencias[modo].append(latencia)
    
    plt.figure(figsize=(10, 6))
    
    for modo, dados in latencias.items():
        ids = list(range(len(dados)))  # Criar uma sequência de 0 a n para o eixo X
        plt.plot(ids, dados, marker='o', label=modo)
    
    plt.xlabel('Número da Requisição')
    plt.ylabel('Latência (time-slots)')
    plt.title('Comparação de Latência entre Modos')
    plt.legend()
    plt.grid(True)
    plt.show()

plotar_latencia(received_requests)